# Walmart Stocks Data Analysis Using Pyspark


Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the 10 years. 

Here is what walmart says about there latest data:
"Our historical data has the Open, High, Low, Close, and volume numbers along with Walmart P/E ratio, and PS ratio. The Walmart stock price history chart shows that the stock price reached a high of 90.47 on 08 Jan, 2015, and a low of 42.27 on 08 Jan, 2015 in last five years."

#### Using the walmart_stock.csv file to Answer and complete the  tasks below!

#### Starting a simple Spark Session

In [1]:
from pyspark.sql import SparkSession 
spark =   SparkSession.builder.appName("Walmart Stock").getOrCreate()

####  Walmart stock

In [2]:
walmart_df = spark.read.csv("WalmartStock_2017.csv",header=True, inferSchema=True)

#### Features or columns

In [3]:
walmart_df.columns

['Date',
 'Open',
 'High',
 'Low',
 'Close',
 'Volume',
 'Price to Earnings Ratio',
 'Price to Sales Ratio']

#### What does the Schema look like?

In [4]:
walmart_df.printSchema()

root
 |-- Date: timestamp (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Price to Earnings Ratio: string (nullable = true)
 |-- Price to Sales Ratio: string (nullable = true)



#### Features Observations

In [5]:
walmart_df.take(5)

[Row(Date=datetime.datetime(2017, 9, 12, 0, 0), Open=79.3, High=79.91, Low=79.17, Close=79.61, Volume=6004802, Price to Earnings Ratio='19.1370', Price to Sales Ratio='0.4887'),
 Row(Date=datetime.datetime(2017, 9, 11, 0, 0), Open=79.15, High=79.35, Low=78.66, Close=79.08, Volume=7000202, Price to Earnings Ratio='19.0100', Price to Sales Ratio='0.4854'),
 Row(Date=datetime.datetime(2017, 9, 8, 0, 0), Open=79.72, High=79.875, Low=77.73, Close=78.88, Volume=9999593, Price to Earnings Ratio='18.9620', Price to Sales Ratio='0.4842'),
 Row(Date=datetime.datetime(2017, 9, 7, 0, 0), Open=80.32, High=80.76, Low=80.0, Close=80.12, Volume=6820634, Price to Earnings Ratio='19.2600', Price to Sales Ratio='0.4918'),
 Row(Date=datetime.datetime(2017, 9, 6, 0, 0), Open=79.88, High=80.4, Low=79.72, Close=80.08, Volume=8425487, Price to Earnings Ratio='19.2500', Price to Sales Ratio='0.4916')]

#### Use describe() to learn about the DataFrame.

In [6]:
walmart_df.describe().show()

+-------+------------------+------------------+------------------+------------------+--------------------+-----------------------+--------------------+
|summary|              Open|              High|               Low|             Close|              Volume|Price to Earnings Ratio|Price to Sales Ratio|
+-------+------------------+------------------+------------------+------------------+--------------------+-----------------------+--------------------+
|  count|              2665|              2665|              2665|              2665|                2665|                   2665|                2665|
|   mean|61.456234521575894|61.933444277673615| 61.01721313320817|63.655750469043035|1.1900423796247656E7|      14.93059052059055|  0.4961338772338774|
| stddev|16.135985131238954|16.196346452770083|16.086215691575212| 11.46608356312984|   7962357.992945932|      1.641863172019808| 0.04618966587390132|
|    min|               0.0|               0.0|               0.0|             42.27|   

In [7]:
walmart_df.describe().printSchema()

root
 |-- summary: string (nullable = true)
 |-- Open: string (nullable = true)
 |-- High: string (nullable = true)
 |-- Low: string (nullable = true)
 |-- Close: string (nullable = true)
 |-- Volume: string (nullable = true)
 |-- Price to Earnings Ratio: string (nullable = true)
 |-- Price to Sales Ratio: string (nullable = true)



In [ ]:
# for better understanding 
# see columns below
result = walmart_df.describe()
result

DataFrame[summary: string, Open: string, High: string, Low: string, Close: string, Volume: string, Price to Earnings Ratio: string, Price to Sales Ratio: string]

In [ ]:
from pyspark.sql.functions import format_number
result.select(result['summary'], result['Volume'].cast('int').alias('Volume'), format_number(result['Open'].cast('float'),2).alias('Open')
             ,format_number(result['High'].cast('float'),2).alias('High'),
             format_number(result['Low'].cast('float'),2).alias('Low'),
             format_number(result['Close'].cast('float'),2).alias('Close'),
              format_number(result['Price to Earnings Ratio'].cast('float'),2).alias('Price to Earnings Ratio'),
              format_number(result['Price to Sales Ratio'].cast('float'),2).alias('Price to Sales Ratio')
             ).show()

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
walmart_df2 = walmart_df.withColumn('Ratio HV',walmart_df['High'] /walmart_df['Volume'])

walmart_df2.show()

#### What day had the Peak High in Price?

In [ ]:
walmart_df.orderBy(walmart_df['High'].desc()).head(1)[0][0]

#### What is the mean of the Close column?

In [ ]:
from pyspark.sql.functions import mean
walmart_df.select(mean(walmart_df["Close"])).show()

#### What is the max and min of the Volume column?

In [ ]:
from pyspark.sql.functions import mean
walmart_df.select(mean(walmart_df["Volume"])).show()

In [ ]:
walmart_df.select(max(walmart_df["Volume"]),max(walmart_df["Volume"])).show()

#### How many days was the Close lower than 60 dollars?

In [ ]:
walmart_df.filter("Close < 60").show()

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
( walmart_df.filter('High > 80').count()/ walmart_df.count())*100

#### What is the Pearson correlation between High and Volume?


In [ ]:
walmart_df.select(corr("High","Volume")).show()

#### What is the max High per year?

In [ ]:
yeardf = walmart_df.withColumn("Year",year(walmart_df["Date"]))
#yeardf.show()
max_df = yeardf.select('High','Close','Date','Year').groupBy('Year').agg(max('High'))
max_df.show()
#Or
#max_df.select('Year','max(High)').show()

#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [ ]:
monthdf = walmart_df.withColumn('month',month('Date'))
monthdf.show()

monthavgs = monthdf.select("Month","Close").groupBy("Month").mean()
monthavgs.select("Month",'avg(Close)').orderBy('Month').show()